In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf

import time

import os
from os import listdir
from os.path import isfile, join

In [2]:
#Some Defatult Values
PATH = "C:/Users/dietd/Untitled Folder/Python for Finance/Financial Data/Stock Data/"
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR,S_MONTH, S_DAY)

E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR,E_MONTH, E_DAY)


In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
#tickers

In [4]:
#function that creates df from csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH+ticker+'.csv')
    except FileNotFoundError:
        print('File Does Not Exist')
    else:
        return df

In [5]:
#function that saves df to csv
def save_df_to_csv(df, ticker):
    df.to_csv(PATH+ticker+'.csv')

In [6]:
#Return valid dates in dataframe
def get_valid_dates(df, sdate, edate):
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate)
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()

        date_leading = '-'.join(('0' if len(x) < 2 else '') + x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x) < 2 else '') + x for x in last_date.split('-'))

    except Exception:
        print("Date is Corrupted")

    else:
        return date_leading, date_ending

In [7]:
#Provide ROI between two dates
def roi_between_dates(df, sdate, edate):
    try:
        start_val = df.loc[sdate, 'Adj Close']
        end_val = df.loc[edate, 'Adj Close']
        roi = ((end_val-start_val)/start_val)

    except Exception:
        print("Data Corrupted")

    else:
        return roi

In [8]:
#get mean, sd, cov
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]['Adj Close'].mean()

def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]['Adj Close'].std()

def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

In [9]:
def get_cov_ror(tickers, sdate, edate):
    col_names = ["Ticker", "COV", "ROI"]
    df = pd.DataFrame(columns=col_names)
    for ticker in tickers:
        print("Working on: ", ticker)
        s_df = get_df_from_csv(ticker)
        dates_period = get_valid_dates(s_df, sdate, edate)
        #print(dates_period)
        start_date, end_date = dates_period
        cov = get_cov_between_dates(s_df, start_date, end_date)
        s_df = s_df.set_index(['Date'])
        roi = roi_between_dates(s_df, start_date, end_date)
        df.loc[len(df.index)] = [ticker, cov, roi]

    return df

In [10]:
#merge multiple stock in one df by column name
def merge_df_by_column_name(col_name, sdate, edate, *tick_args):
    mult_df = pd.DataFrame()
    #print(tick_args[0])
    for x in tick_args:
        #print(x)
        df = get_df_from_csv(x)
        df['Date']=pd.to_datetime(df['Date'])
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
    
    return mult_df
        

In [16]:
#get sectors
sec_df = pd.read_csv("/Users/dietd/Untitled Folder/Python for Finance/Financial Data/stock_sectors.csv")
display(sec_df.shape, sec_df) #S&P500 stocks

indust_df = sec_df.loc[sec_df['Sector']=="Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]


(505, 3)

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [17]:
#Return df with all ROI for all stocks
def get_rois_for_stocks(stock_df):
    tickers = []
    rois = []
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
        if df is None:
            pass
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)

    return pd.DataFrame({'Ticker': tickers, 'ROI': rois})

In [19]:
industrial = get_rois_for_stocks(indust_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)


File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist
File Does Not Exist


In [21]:
display(industrial.sort_values(by=['ROI'], ascending=False).head(5),
        health_care.sort_values(by=['ROI'], ascending=False).head(5),
        it.sort_values(by=['ROI'], ascending=False).head(5),
        commun.sort_values(by=['ROI'], ascending=False).head(5),
        staple.sort_values(by=['ROI'], ascending=False).head(5),
        discretion.sort_values(by=['ROI'], ascending=False).head(5),
        utility.sort_values(by=['ROI'], ascending=False).head(5),
        finance.sort_values(by=['ROI'], ascending=False).head(5),
        material.sort_values(by=['ROI'], ascending=False).head(5),
        restate.sort_values(by=['ROI'], ascending=False).head(5),
        energy.sort_values(by=['ROI'], ascending=False).head(5))



,Ticker,ROI
23,GNRC,3.734749
10,CPRT,1.918578
53,TDG,1.518984
9,CTAS,1.325594
38,ODFL,1.202971


,Ticker,ROI
23,DXCM,5.375582
31,IDXX,2.167543
57,WST,1.887307
47,RMD,1.575014
15,CTLT,1.475499


,Ticker,ROI
20,ENPH,66.488464
2,AMD,7.352459
48,PAYC,4.622203
55,NOW,3.178471
67,ZBRA,2.705814


,Ticker,ROI
11,NFLX,1.689262
15,TMUS,1.105715
4,CHTR,0.895722
16,TTWO,0.840804
10,LYV,0.722861


,Ticker,ROI
10,EL,1.145196
9,COST,1.112863
19,MKC,0.977708
3,CHD,0.838448
25,PG,0.674051


...

,Ticker,ROI
0,AES,1.402914
18,NEE,1.136332
4,AWK,0.819223
26,WEC,0.530405
27,XEL,0.519896


,Ticker,ROI
38,MSCI,2.612068
33,MKTX,1.860333
6,AJG,1.112315
36,MCO,1.044882
47,SPGI,1.012124


,Ticker,ROI
20,SHW,0.824243
12,LIN,0.785526
0,APD,0.771644
15,NEM,0.689577
7,ECL,0.658577


,Ticker,ROI
20,SBAC,0.772695
7,EQIX,0.692928
16,PLD,0.689758
1,AMT,0.685783
5,CCI,0.622156


,Ticker,ROI
8,HES,0.168440
14,OKE,-0.131481
9,KMI,-0.146362
2,COP,-0.217487
19,WMB,-0.218709


In [28]:
#Take best performing stock from each sector
indus_stock =industrial.loc[industrial['ROI'] == industrial['ROI'].max(), 'Ticker'].values[0]
health_stock = health_care.loc[health_care['ROI'] == health_care['ROI'].max(), 'Ticker'].values[0]
it_stock = it.loc[it['ROI'] == it['ROI'].max(), 'Ticker'].values[0]
commun_stock = commun.loc[commun['ROI'] == commun['ROI'].max(), 'Ticker'].values[0]
staple_stock = staple.loc[staple['ROI'] == staple['ROI'].max(), 'Ticker'].values[0]
discret_stock = discretion.loc[discretion['ROI'] == discretion['ROI'].max(), 'Ticker'].values[0]
util_stock = utility.loc[utility['ROI'] == utility['ROI'].max(), 'Ticker'].values[0]
fin_stock = finance.loc[finance['ROI'] == finance['ROI'].max(), 'Ticker'].values[0]
mat_stock = material.loc[material['ROI'] == material['ROI'].max(), 'Ticker'].values[0]
restate_stock = restate.loc[restate['ROI'] == restate['ROI'].max(), 'Ticker'].values[0]
energy_stock = energy.loc[energy['ROI'] == energy['ROI'].max(), 'Ticker'].values[0]

portfolio_list = [indus_stock, health_stock, it_stock, commun_stock, staple_stock, discret_stock, util_stock, fin_stock, mat_stock, restate_stock,energy_stock]
print(portfolio_list)

['GNRC', 'DXCM', 'ENPH', 'NFLX', 'EL', 'TSLA', 'AES', 'MSCI', 'SHW', 'SBAC', 'HES']


In [31]:
#Check correlation
mult_df = merge_df_by_column_name("Daily_Return", '2018-01-01', '2020-12-31', *portfolio_list)
mult_df

mult_corr_matrix = mult_df.corr()

mult_corr_matrix

,GNRC,DXCM,ENPH,NFLX,EL,TSLA,AES,MSCI,SHW,SBAC,HES
GNRC,1.000000,0.289526,0.344129,0.340732,0.418513,0.321048,0.433322,0.515742,0.547848,0.444762,0.426870
DXCM,0.289526,1.000000,0.212156,0.307643,0.263908,0.293396,0.218425,0.469509,0.348673,0.243461,0.243239
ENPH,0.344129,0.212156,1.000000,0.282688,0.256654,0.280869,0.261715,0.341827,0.278754,0.207225,0.299728
NFLX,0.340732,0.307643,0.282688,1.000000,0.293106,0.360827,0.204158,0.449132,0.346570,0.216935,0.217856
EL,0.418513,0.263908,0.256654,0.293106,1.000000,0.277750,0.407448,0.457616,0.506497,0.397506,0.445605
TSLA,0.321048,0.293396,0.280869,0.360827,0.277750,1.000000,0.273606,0.373189,0.344015,0.181884,0.259790
AES,0.433322,0.218425,0.261715,0.204158,0.407448,0.273606,1.000000,0.483399,0.469867,0.333250,0.470412
MSCI,0.515742,0.469509,0.341827,0.449132,0.457616,0.373189,0.483399,1.000000,0.572601,0.498281,0.411647
SHW,0.547848,0.348673,0.278754,0.346570,0.506497,0.344015,0.469867,0.572601,1.000000,0.493168,0.375061
SBAC,0.444762,0.243461,0.207225,0.216935,0.397506,0.181884,0.333250,0.498281,0.493168,1.000000,0.273548


In [32]:
days = len(mult_df.index)

#function that get stock price on certain date
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date, 'Adj Close']

In [50]:
#print("GNRC Stock Price on 2021-01-04: ", get_stock_price_on_date('GNRC', '2021-01-04')) #2
#print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) # 1 357.74
#print("AMD $", get_stock_price_on_date('AMD', '2021-01-04')) # 4 369.2
#print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04')) # 1 522.86
# print("COST $", get_stock_price_on_date('COST', '2021-01-04')) # 1 377.88
# print("TGT $", get_stock_price_on_date('TGT', '2021-01-04')) # 3 527.46
# print("AES $", get_stock_price_on_date('AES', '2021-01-04')) # 18 410.76
# print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) # 1 435.40
# print("NEM $", get_stock_price_on_date('NEM', '2021-01-04')) # 7 434.49
# print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) # 2 548.84
# print("HES $", get_stock_price_on_date('HES', '2021-01-04')) # 9 472.23

#Approximately equal weights
portfolio_stock_prices_day1 = np.array([ get_stock_price_on_date(tick, '2021-01-04') for tick in portfolio_list])
approx_weight_factors = np.array([2, 6,3,1,2,2,25,1,2,2,10])

for i in range(len(portfolio_list)):
    print("Stock price of ", portfolio_list[i], " on 2021-01-04: ", portfolio_stock_prices_day1[i])

print('-'*80)
tot_inv_one_stock = sum([get_stock_price_on_date(tick, '2021-01-04') for tick in portfolio_list])
print('Total investment for one stock each: ', tot_inv_one_stock)

print('-'*80)
tot_price_per_stock = np.array([approx_weight_factors[i]*portfolio_stock_prices_day1[i] for i in range(len(portfolio_list))])
print("Total price per stock for approximate equal weights: ")
for i in range(len(portfolio_list)):
    print(approx_weight_factors[i]," stock of ", portfolio_list[i], ": ", tot_price_per_stock[i])

print('-'*80)
tot_inv = np.dot(portfolio_stock_prices_day1.T, approx_weight_factors)
print("Total investment for (approximately) equally weighted: ", tot_inv)


Stock price of  GNRC  on 2021-01-04:  224.5599975585937
Stock price of  DXCM  on 2021-01-04:  89.43499755859375
Stock price of  ENPH  on 2021-01-04:  172.24000549316406
Stock price of  NFLX  on 2021-01-04:  522.8599853515625
Stock price of  EL  on 2021-01-04:  249.21365356445312
Stock price of  TSLA  on 2021-01-04:  243.2566680908203
Stock price of  AES  on 2021-01-04:  21.768211364746094
Stock price of  MSCI  on 2021-01-04:  428.2048645019531
Stock price of  SHW  on 2021-01-04:  235.72378540039065
Stock price of  SBAC  on 2021-01-04:  269.1463928222656
Stock price of  HES  on 2021-01-04:  51.12406921386719
--------------------------------------------------------------------------------
Total investment for one stock each:  2507.53263092041
--------------------------------------------------------------------------------
Total price per stock for approximate equal weights: 
2  stock of  GNRC :  449.1199951171874
6  stock of  DXCM :  536.6099853515625
3  stock of  ENPH :  516.72001647949

In [51]:
#get weights
def get_portfolio_weight(price, total):
    return price / total

In [52]:

weights = np.array([get_portfolio_weight(approx_weight_factors[i]*portfolio_stock_prices_day1[i], tot_inv) for i in range(len(portfolio_list))])

weights

array([0.08160415, 0.09750089, 0.09388693, 0.09500255, 0.09056318,
       0.08839844, 0.09888094, 0.07780391, 0.08566102, 0.09780665,
       0.09289135])

In [56]:
portfolio_covar = np.dot(weights.T, np.dot(mult_df.cov()*days, weights))
print("Portfolio Covariance: ", portfolio_covar)
print("GNRC Variance: ", mult_df['GNRC'].var()*days)

Portfolio Covariance:  0.25727359758266655
GNRC Variance:  0.43946543607242516


In [57]:
#Different types of risk

#Diversifiable risk 
def calc_diversifiable_risk(df, tickers, weights):
    days = len(df.index)
    port_covar = np.dot(weights.T, np.dot(df.cov()*days, weights))
    i = 0
    while i < len(tickers):
        weight_sq = weights[i] ** 2
        stock_var = df[tickers[i]].var() * days
        weight_var = weight_sq * stock_var
        port_covar = port_covar - weight_var
        i += 1

    return port_covar

In [58]:
div_risk = calc_diversifiable_risk(mult_df, portfolio_list, weights)
div_risk

0.18862773889060694

In [60]:
#Systematic risk
print("Systematic Risk: ", portfolio_var - div_risk)

Systematic Risk:  0.06864585869205961
